In [48]:
import talib
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [49]:
 def check_body(df):
        #     Calculate 0.7 % of entry price
    #     fOR 1H FOREX
            one_percent = df['open']/100 
            percent = one_percent * 0.06
            hammer_per = one_percent * 0.06

#             print('Percent: ', percent)
        #     Calculate candle size close-open
            if df['candle'] == 'G':
                candle_size = df['close'] - df['open']
                upper_wick = df['high'] - df['close']
                lower_wick = df['open'] - df['low']

            else:
                candle_size = abs(df['close'] - df['open'])
                upper_wick = df['high'] - df['open']
                lower_wick = df['close'] - df['low']

            upper_wick_per = (upper_wick/one_percent) 
            lower_wick_per = (lower_wick/one_percent) 


            if (candle_size >= percent):
        #     check for entry candle size and Max candle size 0.12 %
                if (candle_size <= (0.12 * one_percent) and
                 (candle_size > upper_wick) and (candle_size > lower_wick) and
                (upper_wick_per < 0.06) and (lower_wick_per < 0.06)):

                    if df['candle'] == 'G' :

                        return 1

                    elif df['candle'] == 'R':

                        return 1

                    else:

                        return 0

            #     condition for hammmer
            #     lower wick is atleast 50 % of total size 
            #     Green Hammer 
            # need to add max wick of opposite dir
                elif ( (df['candle'] == 'G') and (df['open'] - df['low']) >= ((df['high'] - df['low']) * 0.5) and
                      (candle_size >= hammer_per) and (candle_size > upper_wick)and
                      candle_size <= (0.12 * one_percent) and ((upper_wick_per < 0.06)) ):

                    return 1


        #     For Red Hammer
        # need to add max wick of opposite dir
                elif ( (df['candle'] == 'R') and (df['high'] - df['open']) >= (abs(df['high'] - df['low']) * 0.5) and 
                      (candle_size >= hammer_per) and (candle_size > lower_wick)and candle_size <= (0.12 * one_percent) and 
                      ((lower_wick_per < 0.06)) ):

                    return 1


                else:
                    return 0
                
            return 0



In [50]:
def candle(df):
        '''Return candle colour'''

        if df['close'] - df['open'] > 0:
            return 'G'

        else:
            return 'R'

In [51]:
# if not mt5.initialize(login=40987, server="CabanaCapitals-Demo",password="Mypassword$1234"):
#             print("initialize() failed, error code =",mt5.last_error())
#             quit()
            
mt5.initialize(login=40987, server="CabanaCapitals-Demo",password="Mypassword$1234")
        
login = 40987
password = 'Mypassword$1234'
server = 'CabanaCapitals-Demo'
mt5.login(login, password, server)

rate = mt5.copy_rates_from('GBPUSD', mt5.TIMEFRAME_M15, datetime.datetime.now(), 1000)
df = pd.DataFrame(rate)
#         print('getting data')

df['date']=pd.to_datetime(df['time'], unit='s')

df.set_index(np.arange(len(df)), inplace = True)
df = df.drop(['spread', 'real_volume', 'tick_volume', 'time'], axis='columns')

df['prev_candle'] = False
df['candle'] = False


for index, row in df.iterrows():
#     print(row)
    df.loc[index, 'candle'] = candle(row)
#     print(df.iloc[index])
#     print()
    df.loc[index, 'prev_candle'] = check_body(df.iloc[index])
#     print(df.iloc[index])
    
    
df.to_excel('review.xlsx')
    

PermissionError: [Errno 13] Permission denied: 'review.xlsx'